<a id="top"></a>
# Query a region, plot an HR diagram, and select a light curve: Solutions
***

## Imports

In [2]:
#import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from astropy.coordinates import SkyCoord
from astroquery.mast import Catalogs
from astroquery.mast import Observations
import bokeh
from bokeh.models.tools import *
from bokeh.plotting import *
#from bokeh.models import CustomJS, TapTool, ColumnDataSource, Row
#from bokeh.models.widgets import Button
#from random import choice
#from string import ascii_lowercase
#from bokeh.io import push_notebook, show, output_notebook, curdoc
import astropy.units as u

#from astropy.io import fits
from astropy.table import Table

from astroquery.mast import Observations
#from astroquery.mast import Tesscut

import lightkurve as lk

%matplotlib inline

## Exercise 1: 

What percentage of the returned results have...  
a) ...distance measurements included in the TIC?  
b) ...measurements for ``gaiabp`` *and* ``gaiarp``?   
c) ... measurements for ``Bmag`` *and* ``Vmag``?   

In [3]:
# First, we need to recreate some of the variables from the original notebook here in the solutions notebook.
# select a region of the sky to query
ra = 180.
dec = +30.
radius=1*u.degree
coord = SkyCoord(ra, dec, unit="deg")
catalog_data = Catalogs.query_region(coord, radius=radius, catalog="Tic")

In [4]:
# Solution:
n=len(catalog_data)
print('Total length: ',n)
print('Percentage with non-nan d:',np.sum(~np.isnan(catalog_data['d']))/n*100)
print('Percentage with gaiabp and gaiarp:',np.sum(~np.isnan(catalog_data['gaiabp']) & ~np.isnan(catalog_data['gaiarp']))/n*100)
print('Percentage with Bmag and Vmag:',np.sum(~np.isnan(catalog_data['Bmag']) & ~np.isnan(catalog_data['Vmag']))/n*100)

Total length:  8774
Percentage with non-nan d: 67.15295190335081
Percentage with gaiabp and gaiarp: 70.86847503989058
Percentage with Bmag and Vmag: 28.037383177570092


## Exercise 2
Using the hover tool, what are the TIC identifiers numbers of stars at the following locations?  
a) (Gb-Gr,G) = (0.865, 0.459)  
b) (Gb-Gr,G) = (1.691, 7.024)  
c) (Gb-Gr,G) = (-0.0288,11.869)  
Confirm your choices by checking the catalog.
Based on their locations in the HR diagram, what type of stars are each of these likely to be?
If you wanted to search for exoplanets orbiting stars, which of the three stars would you investigate further?
If you wanted to examine stellar variability, which of the three stars would you investigate further? 

In [8]:
# First, we need to recreate some of the variables from the original notebook here in the solutions notebook.
catalog_data = catalog_data[(catalog_data['d'] > 0) & (catalog_data['e_d'] / catalog_data['d'] < 0.1)]
catalog_data['abs_gaia'] = catalog_data['GAIAmag'] - 5 * np.log10(catalog_data['d'] / 10)
catalog_data['color'] = catalog_data['gaiabp'] - catalog_data['gaiarp']
catalog_data=catalog_data[~np.isnan(catalog_data['color'])]
catalog_data=catalog_data[~np.isnan(catalog_data['abs_gaia'])]

In [9]:
# Solution:
# Based on hovering, the TIC ID's are: a) 138898766, b) 173091222, c) 903745613
print(catalog_data[catalog_data['ID']=='138898766']['ID','color','abs_gaia','lumclass','Teff'])
print(catalog_data[catalog_data['ID']=='173091222']['ID','color','abs_gaia','lumclass','Teff'])
print(catalog_data[catalog_data['ID']=='903745613']['ID','color','abs_gaia','lumclass','Teff'])
# The first star is a giant, the second is on the main sequence, and the third is likely a stellar remnant (white dwarf).
# The star on the main sequence is the best one to investigate for exoplanets.
# The giant star is the best one to investigate for variability, as it is likely on the instability strip.

    ID          color             abs_gaia     lumclass  Teff 
--------- ------------------ ----------------- -------- ------
138898766 0.8646000000000011 0.459238477818106    GIANT 5717.0
    ID          color             abs_gaia      lumclass  Teff 
--------- ------------------ ------------------ -------- ------
173091222 1.6906000000000017 7.0242717177156955    DWARF 3932.4
    ID           color              abs_gaia      lumclass Teff
--------- -------------------- ------------------ -------- ----
903745613 -0.02880000000000038 11.869260443380862    DWARF  nan


## Exercise about which ones have LC

In [10]:
catalog_data['HasLC']=False
# For my purposes... how many have LC's
for i,row in enumerate(catalog_data):
    search_result=lk.search_lightcurve('TIC '+str(row['ID']))
    if len(search_result)>0:
        catalog_data['HasLC'][i]=True
print(np.sum(catalog_data['HasLC']))
catalog_haslc=catalog_data[catalog_data['HasLC']]
catalog_haslc.write('hr_haslc.csv',format='csv')

No data found for target "TIC 173047644".
No data found for target "TIC 173047636".
No data found for target "TIC 173047632".
No data found for target "TIC 173047625".
No data found for target "TIC 173047666".
No data found for target "TIC 138932666".
No data found for target "TIC 173047627".
No data found for target "TIC 173047677".
No data found for target "TIC 173047664".
No data found for target "TIC 173047645".
No data found for target "TIC 173047643".
No data found for target "TIC 138932670".


KeyboardInterrupt: 